### 환자의 진단과 처방을 한 테이블을 만들기 위한 노트북입니다. 
used table : ADDMISSION | DIAGNOSES_ICD | D_ICD_DIAGNOSES | PROCEDURES_ICD | D_ICD_PROCEDURES

In [165]:
import pandas as pd

In [166]:
BASE_DIR = 'data/'

addmission  = pd.read_csv(BASE_DIR + 'ADMISSIONS.csv')

diagnoses   = pd.read_csv(BASE_DIR + 'DIAGNOSES_ICD.csv')
procedures  = pd.read_csv(BASE_DIR + 'PROCEDURES_ICD.csv')

dict_diagnoses  = pd.read_csv(BASE_DIR + 'D_ICD_DIAGNOSES.csv')
dict_procedures = pd.read_csv(BASE_DIR + 'D_ICD_PROCEDURES.csv')

addmission_diag = addmission[['ROW_ID', 'SUBJECT_ID', 'DIAGNOSIS']]


# addmission_diag.info()
# print(addmission_diag['ICD9_CODE'].duplicated(keep=False).sum())
'''
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ROW_ID      58976 non-null  int64 
 1   SUBJECT_ID  58976 non-null  int64 
 2   DIAGNOSIS   58951 non-null  object
'''
print(diagnoses['ICD9_CODE'].duplicated(keep=False).sum())
# diagnoses.info()
'''
#   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651000 non-null  object 
'''
# procedures.info()
print(procedures['ICD9_CODE'].duplicated(keep=False).sum())

'''
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   ROW_ID      240095 non-null  int64
 1   SUBJECT_ID  240095 non-null  int64
 2   HADM_ID     240095 non-null  int64
 3   SEQ_NUM     240095 non-null  int64
 4   ICD9_CODE   240095 non-null  int64
'''
# dict_diagnoses.info()
print(dict_diagnoses['ICD9_CODE'].duplicated(keep=False).sum())
'''
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       14567 non-null  int64 
 1   ICD9_CODE    14567 non-null  object
 2   SHORT_TITLE  14567 non-null  object
 3   LONG_TITLE   14567 non-null  object
'''
# dict_procedures.info()
print(dict_procedures['ICD9_CODE'].duplicated(keep=False).sum())
'''
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       3882 non-null   int64 
 1   ICD9_CODE    3882 non-null   int64 
 2   SHORT_TITLE  3882 non-null   object
 3   LONG_TITLE   3882 non-null   object
'''

649544
239705
0
146


'\n #   Column       Non-Null Count  Dtype \n---  ------       --------------  ----- \n 0   ROW_ID       3882 non-null   int64 \n 1   ICD9_CODE    3882 non-null   int64 \n 2   SHORT_TITLE  3882 non-null   object\n 3   LONG_TITLE   3882 non-null   object\n'

In [167]:
addmission_diag

,ROW_ID,SUBJECT_ID,DIAGNOSIS
0,21,22,BENZODIAZEPINE OVERDOSE
1,22,23,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
2,23,23,BRAIN MASS
3,24,24,INTERIOR MYOCARDIAL INFARCTION
4,25,25,ACUTE CORONARY SYNDROME
...,...,...,...
58971,58594,98800,TRAUMA
58972,58595,98802,SAH
58973,58596,98805,RENAL CANCER/SDA
58974,58597,98813,S/P FALL


In [168]:
diag_ICD = pd.merge(addmission_diag['SUBJECT_ID'], diagnoses[['SUBJECT_ID','ICD9_CODE']], how='outer', on='SUBJECT_ID')
diag_ICD

,SUBJECT_ID,ICD9_CODE
0,22,9678
1,22,9693
2,22,E9502
3,22,E9503
4,22,3488
...,...,...
1374974,98805,27650
1374975,98805,E8786
1374976,98805,7533
1374977,98805,2767


In [169]:
patient_diag_ICD = pd.merge(diag_ICD, dict_diagnoses.drop('ROW_ID',axis=1), how='inner', on='ICD9_CODE') # 둘 다 있는 경우만
patient_diag_ICD = patient_diag_ICD.drop_duplicates()
# patient_diag_ICD = patient_diag_ICD.set_index('SUBJECT_ID')
patient_diag_ICD.sort_values('SUBJECT_ID')

,SUBJECT_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
236637,2,V290,NB obsrv suspct infect,Observation for suspected infectious condition
242283,2,V053,Need prphyl vc vrl hepat,Need for prophylactic vaccination and inoculat...
690459,2,V3001,Single lb in-hosp w cs,"Single liveborn, born in hospital, delivered b..."
155352,3,41071,"Subendo infarct, initial","Subendocardial infarction, initial episode of ..."
279323,3,0389,Septicemia NOS,Unspecified septicemia
...,...,...,...,...
901990,99999,7861,Stridor,Stridor
154721,99999,25000,DMII wo cmp nt st uncntr,Diabetes mellitus without mention of complicat...
657968,99999,4139,Angina pectoris NEC/NOS,Other and unspecified angina pectoris
1332659,99999,75612,Spondylolisthesis,Spondylolisthesis


In [170]:
diagnoses_ICD = diagnoses.drop('ROW_ID', axis=1)
diagnoses_ICD = diagnoses_ICD.rename(columns={'ICD9_CODE': 'diag_ICD'})
procedures_ICD = procedures.drop('ROW_ID', axis=1)
procedures_ICD = procedures_ICD.rename(columns={'ICD9_CODE': 'proc_ICD'})

In [171]:
merged_df = pd.merge(diagnoses_ICD, procedures_ICD, 
                     on=['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM'], how='left')
merged_df = merged_df.dropna()
merged_df

,SUBJECT_ID,HADM_ID,SEQ_NUM,diag_ICD,proc_ICD
0,109,172335,1.0,40301,5498.0
14,109,173633,1.0,40301,3995.0
15,109,173633,2.0,5856,3893.0
16,109,173633,3.0,58381,9904.0
28,112,174105,1.0,53100,4443.0
...,...,...,...,...,...
651032,97497,168949,1.0,0529,9969.0
651033,97497,168949,2.0,4162,9929.0
651034,97497,168949,3.0,20192,9925.0
651041,97503,188195,1.0,7842,260.0


In [172]:
dict_procedures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3882 entries, 0 to 3881
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       3882 non-null   int64 
 1   ICD9_CODE    3882 non-null   int64 
 2   SHORT_TITLE  3882 non-null   object
 3   LONG_TITLE   3882 non-null   object
dtypes: int64(2), object(2)
memory usage: 121.4+ KB


In [173]:
merged_df = pd.merge(merged_df, 
                    dict_diagnoses[['ICD9_CODE','SHORT_TITLE','LONG_TITLE']], 
                    how='inner', 
                    left_on='diag_ICD',
                    right_on='ICD9_CODE')
merged_df = merged_df.rename(columns={'SHORT_TITLE': 'SHORT_TITLE_diag',
                                      'LONG_TITLE': 'LONG_TITLE_diag',})

merged_df = merged_df.drop('ICD9_CODE', axis=1)

merged_df = pd.merge(merged_df, 
                    dict_procedures[['ICD9_CODE','SHORT_TITLE','LONG_TITLE']], 
                    how='inner', 
                    left_on='proc_ICD',
                    right_on='ICD9_CODE')


merged_df = merged_df.rename(columns={'SHORT_TITLE': 'SHORT_TITLE_proc',
                                      'LONG_TITLE': 'LONG_TITLE_proc',})
merged_df = merged_df.drop('ICD9_CODE', axis=1)
merged_df = merged_df.sort_values('SUBJECT_ID')
merged_df = merged_df.set_index('SUBJECT_ID')
merged_df[['SEQ_NUM','proc_ICD']] = merged_df[['SEQ_NUM','proc_ICD']].astype('int32')
merged_df

,HADM_ID,SEQ_NUM,diag_ICD,proc_ICD,SHORT_TITLE_diag,LONG_TITLE_diag,SHORT_TITLE_proc,LONG_TITLE_proc
SUBJECT_ID,,,,,,,,
2,163353,1,V3001,9955,Single lb in-hosp w cs,"Single liveborn, born in hospital, delivered b...",Vaccination NEC,Prophylactic administration of vaccine against...
3,145834,2,78559,9962,Shock w/o trauma NEC,Other shock without mention of trauma,Heart countershock NEC,Other electric countershock of heart
3,145834,3,5849,8964,Acute kidney failure NOS,"Acute kidney failure, unspecified",Pulmon art wedge monitor,Pulmonary artery wedge monitoring
3,145834,5,41071,3893,"Subendo infarct, initial","Subendocardial infarction, initial episode of ...",Venous cath NEC,"Venous catheterization, not elsewhere classified"
3,145834,4,4275,9672,Cardiac arrest,Cardiac arrest,Cont inv mec ven 96+ hrs,Continuous invasive mechanical ventilation for...
...,...,...,...,...,...,...,...,...
99999,113369,3,4019,8162,Hypertension NOS,Unspecified essential hypertension,Fus/refus 2-3 vertebrae,Fusion or refusion of 2-3 vertebrae
99999,113369,5,4139,8451,Angina pectoris NEC/NOS,Other and unspecified angina pectoris,Ins spinal fusion device,Insertion of interbody spinal fusion device
99999,113369,1,75612,8108,Spondylolisthesis,Spondylolisthesis,Lumb/lmbsac fus ant/post,Lumbar and lumbosacral fusion of the anterior ...


In [174]:
merged_df.to_csv(BASE_DIR+'PAIENT_DIAG_PROC.csv')